In [9]:
import random
import itertools
import numpy
import math

In [3]:
def calcular_matriz_distancias(dataset):
    n_ciudades = len(dataset)
    distancias = numpy.zeros((n_ciudades,n_ciudades))
    for i in range(n_ciudades):
        for j in range(n_ciudades):
            if(i != j):
                distancias[i][j] = distancia_euclidea(dataset[i],dataset[j])
    return distancias

In [4]:
def distancia_euclidea(ciudad_1,ciudad_2):
    
    x1,y1 = ciudad_1[1 :]
    x2,y2 = ciudad_2[1 :]
    
    xd = x1 - x2;
    yd = y1 - y2;
    
    return round(math.sqrt( xd*xd + yd*yd ))

In [5]:
def calcular_coste(camino,distancias):
    coste = 0
    for indice in range(len(camino)-1):
        coste += distancias[camino[indice]][camino[indice+1]]
    coste += distancias[camino[-1]][camino[0]]
    return coste

In [14]:
def busqueda_local_el_primer_mejor_vecino(semilla, dataset):
    # Se establece la semilla inicial
    random.seed(semilla)

    # Tamaño dataset
    len_dataset = len(dataset)

    # Se obtiene la matriz de distancias
    distancias = calcular_matriz_distancias(dataset)

    # Generación de solución inicial
    solucion = random.sample(range(len_dataset),len_dataset)

    # Generación del coste inicial
    coste = calcular_coste(solucion, distancias)

    # Primera condición de parada del bucle
    num_evaluaciones = len_dataset*1600

    # Todos los posibles intercambios para generar a todos los vecinos
    combinaciones_camino = list(itertools.combinations(list(range(0, len_dataset)), 2))

    # Tamaño del numero de vecinos
    len_combinaciones_camino = len(combinaciones_camino)

    # Indices nodos cercanos
    nodos_cercanos = []
    for vecino in combinaciones_camino:
        if (vecino[0] == 0):
            a1 = len_dataset - 1
        else:
            a1 = vecino[0] - 1

        if (vecino[0] == len_dataset - 1):
            s1 = 0
        else:
            s1 = vecino[0] + 1

        if (vecino[1] == 0):
            a2 = len_dataset - 1
        else:
            a2 = vecino[1] - 1

        if (vecino[1] == len_dataset - 1):
            s2 = 0
        else:
            s2 = vecino[1] + 1

        nodos_cercanos.append([a1, s1, a2, s2])

    ## Variable de salida del bucle
    seguir = True

    # Nº de evaluaciones del bucle
    evaluaciones = 0

    while (seguir):

        # Solución de la iteración, en cada vuelta se inicializa a vacia y si al final del bucle sigue vacia, se termina la ejecución
        solucion_candidata = ()
        coste_candidato = coste

        for i in range(len_combinaciones_camino):

            # Por cada vecino, estudiaremos su coste
            coste_actual = coste

            # Primero, por comodidad, generaremos los indices del camino
            a1 = nodos_cercanos[i][0]
            s1 = nodos_cercanos[i][1]
            a2 = nodos_cercanos[i][2]
            s2 = nodos_cercanos[i][3]
            i1 = combinaciones_camino[i][0]
            i2 = combinaciones_camino[i][1]

            if (i2 - 1 == i1):

                # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

                coste_actual -= distancias[solucion[a1]][solucion[i1]]
                coste_actual -= distancias[solucion[i2]][solucion[s2]]

                # Despues sumamos el coste del nuevo camino

                coste_actual += distancias[solucion[a1]][solucion[i2]]
                coste_actual += distancias[solucion[i1]][solucion[s2]]

            elif(i1 == 0 and i2 == len_dataset-1):
                # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

                coste_actual -= distancias[solucion[a2]][solucion[i2]]
                coste_actual -= distancias[solucion[i1]][solucion[s1]]

                # Despues sumamos el coste del nuevo camino

                coste_actual += distancias[solucion[a2]][solucion[i1]]
                coste_actual += distancias[solucion[i2]][solucion[s1]]
            else:

                # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

                coste_actual -= distancias[solucion[a1]][solucion[i1]]
                coste_actual -= distancias[solucion[i1]][solucion[s1]]

                coste_actual -= distancias[solucion[a2]][solucion[i2]]
                coste_actual -= distancias[solucion[i2]][solucion[s2]]

                # Despues sumamos el coste del nuevo camino

                coste_actual += distancias[solucion[a1]][solucion[i2]]
                coste_actual += distancias[solucion[i2]][solucion[s1]]

                coste_actual += distancias[solucion[a2]][solucion[i1]]
                coste_actual += distancias[solucion[i1]][solucion[s2]]

            evaluaciones += 1
            # Y por ultimo, comprobamos que el nuevo coste es mejor que el candidato a modificar
            if (coste_actual < coste_candidato):
                coste_candidato = coste_actual
                solucion_candidata = combinaciones_camino[i]
                break #La unica modificación respecto al algoritmo Búsqueda local, el mejor

            if evaluaciones >= num_evaluaciones:
                seguir = False
                
        if solucion_candidata == ():
            seguir = False
        else:
            buffer = solucion[solucion_candidata[0]]
            solucion[solucion_candidata[0]] = solucion[solucion_candidata[1]]
            solucion[solucion_candidata[1]] = buffer
            coste = coste_candidato

        if evaluaciones >= num_evaluaciones:
            seguir = False

    return [dataset[ciudad] for ciudad in solucion], coste, evaluaciones